## Spatio_Temporal Train Delay Predictive Model - By Anubhav Kesari

In [376]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-dataset/train.csv
/kaggle/input/train-dataset/test.csv


In [377]:
import time 
import re
import datetime
import math
import plotly
import matplotlib.pyplot as plt
import warnings

In [378]:
%matplotlib inline
warnings.filterwarnings("ignore")

In [379]:
base_dir = "/kaggle/input/train-dataset/"
train_file = base_dir+"train.csv"
test_file = base_dir+"test.csv"

In [380]:
#Basic Dataframe read
df = pd.read_csv(train_file)
df2 =pd.read_csv(test_file)

In [381]:
print(df2.columns)
print(df.columns)

Index(['runDate', 'stations', 'trainId', 'trainCode', 'index',
       'trainStationId', 'scheduledArrival', 'scheduledDeparture', 'distance',
       'dayCount', 'day'],
      dtype='object')
Index(['runDate', 'stations', 'trainCode', 'trainStationId',
       'scheduledArrival', 'scheduledDeparture', 'actualArrival',
       'actualDeparture', 'distance', 'dayCount', 'ArrivalDelay',
       'DepartureDelay'],
      dtype='object')


In [382]:
df.dtypes

runDate               object
stations              object
trainCode              int64
trainStationId         int64
scheduledArrival      object
scheduledDeparture    object
actualArrival         object
actualDeparture       object
distance               int64
dayCount               int64
ArrivalDelay           int64
DepartureDelay         int64
dtype: object

In [383]:
set(df.runDate.tolist())

{'2020-01-01',
 '2020-01-02',
 '2020-01-03',
 '2020-01-04',
 '2020-01-05',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10',
 '2020-01-11',
 '2020-01-12',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-18',
 '2020-01-19',
 '2020-01-20',
 '2020-01-21',
 '2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03',
 '2020-02-04',
 '2020-02-05',
 '2020-02-06',
 '2020-02-07',
 '2020-02-08',
 '2020-02-09',
 '2020-02-10',
 '2020-02-11',
 '2020-02-12',
 '2020-02-13',
 '2020-02-14',
 '2020-02-15',
 '2020-02-16',
 '2020-02-17',
 '2020-02-18',
 '2020-02-19'}

In [384]:
set(df2.runDate.tolist())

{'2020-02-19', '2020-02-20'}

In [385]:
#stationId vs station code
train_station_map=df[['stations','trainStationId']]
train_station = train_station_map.drop_duplicates()

In [386]:
#trainStation Name vs Code
train_station=dict(zip(train_station.stations.tolist() , train_station.trainStationId.tolist()))
# train_station

In [387]:
from collections import Counter
#rush at each station
#station vs trains through it

def station_traffic(df):
    stations = Counter(df.stations.tolist())
#     print(stations)
    return stations

In [388]:
stn_traffic = station_traffic(df)

In [389]:
#I thought of defining a characterstics to take into account the category of train - Superfast ,Express,loca
def train_characterstics(df):
    return 1

# train_characterstics(df)

In [390]:

def process_datetime(x):
    """return utc timestamp corresponding to datetime"""
#     print(x)
    if not isinstance(x,int) :
#         print(x)
        x=datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
        return int(x.timestamp())
    else:
        return x

In [391]:
print(datetime.datetime.strptime("2020-01-01 22:56:00","%Y-%m-%d %H:%M:%S").timestamp())


1577919360.0


## Missing Data treatment 

### Does the missing scheduledArrival because of the fact that it starts at that place?

In [392]:
for i in set(df.trainCode.tolist()):
    a=df[(df['trainCode']==i) & (df['scheduledArrival'].isna())].shape
    b=df[(df['trainCode']==i)].groupby('runDate').mean().shape
    print(a[0]==b[0])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


### Hence, we can impute scheduledDeparture to fill scheduledArrival

In [393]:
#No delay in departure at starting station
df.loc[df['scheduledArrival'].isna(), 'scheduledArrival'] = df['actualArrival']

In [394]:
# df[(df['trainCode']==12925) & (df['scheduledDeparture'].isna())]

In [395]:
df = df[df['scheduledDeparture'].notna()]

In [396]:
#converting datetimes to utc integer
df['actualArrival']= df['actualArrival'].apply(lambda x:process_datetime(x))
df['actualDeparture']=df['actualDeparture'].apply(lambda x:process_datetime(x))
df['scheduledArrival']=df['scheduledArrival'].apply(lambda x:process_datetime(x))
df['scheduledDeparture']=df['scheduledDeparture'].apply(lambda x:process_datetime(x))

In [397]:
# df[(df['trainCode_n']==12138)]

In [398]:
# df2

In [399]:
# df2['scheduledArrival']=df2['scheduledArrival'].apply(lambda x:process_datetime(x))
# df2['scheduledDeparture']=df2['scheduledDeparture'].apply(lambda x:process_datetime(x))

In [400]:
def utc2day(t):
    day = time.strftime('%A', time.localtime(t))
    return day

def utc2hour(t):
    hour = time.strftime('%H', time.localtime(t))
    return hour

In [401]:
#creating a column for knowing the day of the week
# df['day']=df['scheduledArrival'].apply(lambda x : utc2day(x))
# df['hour']=df['scheduledArrival'].apply(lambda x: utc2hour(x))

In [402]:
df

,runDate,stations,trainCode,trainStationId,scheduledArrival,scheduledDeparture,actualArrival,actualDeparture,distance,dayCount,ArrivalDelay,DepartureDelay
0,2020-01-01,FZR,12138,2657812,1577836800,1577914800,1577836800,1577914800,0,0,0,0
1,2020-01-01,FDK,12138,2659393,1577916300,1577916420,1577916960,1577917080,32,0,11,11
2,2020-01-01,KKP,12138,2658108,1577917380,1577917500,1577918220,1577918280,45,0,14,13
3,2020-01-01,GJUT,12138,2734206,1577918280,1577918400,1577919360,1577919420,61,0,18,17
4,2020-01-01,GNA,12138,2742572,1577919240,1577919360,1577920440,1577920500,76,0,20,19
...,...,...,...,...,...,...,...,...,...,...,...,...
31869,2020-02-19,LDH,12925,2658435,1582217160,1582217460,1582221300,1582221840,1687,1,69,73
31870,2020-02-19,PGW,12925,2658284,1582219140,1582219260,1582223940,1582224060,1723,1,80,80
31871,2020-02-19,JRC,12925,2657701,1582220220,1582220340,1582225440,1582225560,1739,1,87,87
31872,2020-02-19,JUC,12925,2657676,1582221180,1582221480,1582226100,1582227000,1744,1,82,92


In [403]:
#add time_superscript
for i in df.columns:
    if  i not in ['runDate','trainStationId']:
        df['{}_n'.format(i)]=df[i]
        del df[i]

In [404]:
del df['trainStationId']

In [405]:
#Convertng time delays given in seconds
df['DepartureDelay_n'] = df['DepartureDelay_n'].apply(lambda x: x*60)
df['ArrivalDelay_n'] = df['ArrivalDelay_n'].apply(lambda x: x*60)

## Basic EDA and Observations

In [406]:
df.head(3)

,runDate,stations_n,trainCode_n,scheduledArrival_n,scheduledDeparture_n,actualArrival_n,actualDeparture_n,distance_n,dayCount_n,ArrivalDelay_n,DepartureDelay_n
0,2020-01-01,FZR,12138,1577836800,1577914800,1577836800,1577914800,0,0,0,0
1,2020-01-01,FDK,12138,1577916300,1577916420,1577916960,1577917080,32,0,660,660
2,2020-01-01,KKP,12138,1577917380,1577917500,1577918220,1577918280,45,0,840,780


#### Spatial Factor 
### Does stations' rush have a role to play in delay ?

In [407]:
df.columns

# df[df['stations_n']=='ADH']['ArrivalDelay_n'].mean()

temp_df = df[['stations_n','ArrivalDelay_n','DepartureDelay_n']].groupby('stations_n').mean().reset_index()
# temp_df

stn_frame = pd.Series(stn_traffic).to_frame().reset_index()
stn_frame = stn_frame.rename(columns={'index':'stations_n',0:'frequency'})

# 'stations_n' in stn_frame.columns

temp_df=pd.merge(stn_frame,temp_df,on='stations_n')

print(temp_df['frequency'].corr(temp_df['DepartureDelay_n']))
print(temp_df['frequency'].corr(temp_df['ArrivalDelay_n']))

0.019912110406907334
0.002589551087336086


#### **LOW** Correlation

### Does the delay in previous stations affect the future delay ? Cross Correlation

In [408]:
temp_df=temp_df[['stations_n','ArrivalDelay_n','DepartureDelay_n']]
temp_df['ArrivalDelay_n+1']=temp_df['ArrivalDelay_n'].shift(1)
temp_df['DepartureDelay_n+1']=temp_df['DepartureDelay_n'].shift(1)
print(temp_df['ArrivalDelay_n+1'].corr(temp_df['ArrivalDelay_n']))
print(temp_df['ArrivalDelay_n+1'].corr(temp_df['ArrivalDelay_n+1']))

0.8621850455496066
1.0


#### Yes , highly related ==> A time series problem

### Does the delay for a particular train depends on day of week ?

In [409]:
#Train No. 12138
import plotly.express as px
# df = px.data.gapminder().query("continent=='Oceania'")
# df

In [410]:
def station_wise_delay_of_train():
    temp_df=df[df['trainCode_n']==12138]
    temp_df['day_n']=temp_df['scheduledArrival_n'].apply(lambda x: utc2day(x))
    temp_df=temp_df[['ArrivalDelay_n','day_n','trainCode_n','stations_n']]
    temp_df.groupby('day_n').mean().reset_index()
#     print(temp_df)
#     print('day_n' in temp_df.columns)
#     print(temp_df.shape)
    fig = px.scatter(temp_df, x="stations_n", y="ArrivalDelay_n", color='day_n')
    fig.show()

In [411]:
station_wise_delay_of_train()

### Does the delay of this train instance depends on previous running train instance

In [412]:
# df[(df['trainCode_n']==12925) &( df['stations_n']=='JUC')]

In [413]:
temp_df=df[df['trainCode_n']==12138]
temp_df=temp_df[['stations_n','ArrivalDelay_n','runDate','trainCode_n']]
train1=temp_df[0:53]
train2=temp_df[53:106]
print(train1)
train1['ArrivalDelay_n'].corr(train2['ArrivalDelay_n'])

   stations_n  ArrivalDelay_n     runDate  trainCode_n
0         FZR               0  2020-01-01        12138
1         FDK             660  2020-01-01        12138
2         KKP             840  2020-01-01        12138
3        GJUT            1080  2020-01-01        12138
4         GNA            1200  2020-01-01        12138
5         BTI             360  2020-01-01        12138
6        MAUR             660  2020-01-01        12138
7         MSZ             900  2020-01-01        12138
8         BLZ             900  2020-01-01        12138
9         BRZ             900  2020-01-01        12138
10        JHL             540  2020-01-01        12138
11        TUN             660  2020-01-01        12138
12        NRW             660  2020-01-01        12138
13       JIND             180  2020-01-01        12138
14        ROK            -120  2020-01-01        12138
15        BGZ              60  2020-01-01        12138
16        SSB           -1200  2020-01-01        12138
17        

nan

### Red Dots are in good number at the bottom 


In [414]:
df.head(2)

,runDate,stations_n,trainCode_n,scheduledArrival_n,scheduledDeparture_n,actualArrival_n,actualDeparture_n,distance_n,dayCount_n,ArrivalDelay_n,DepartureDelay_n
0,2020-01-01,FZR,12138,1577836800,1577914800,1577836800,1577914800,0,0,0,0
1,2020-01-01,FDK,12138,1577916300,1577916420,1577916960,1577917080,32,0,660,660


In [415]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# temp_df=df
le=LabelEncoder()
# df['day_n'] =le.fit_transform(df['day_n'])

In [416]:
df.head(3)

,runDate,stations_n,trainCode_n,scheduledArrival_n,scheduledDeparture_n,actualArrival_n,actualDeparture_n,distance_n,dayCount_n,ArrivalDelay_n,DepartureDelay_n
0,2020-01-01,FZR,12138,1577836800,1577914800,1577836800,1577914800,0,0,0,0
1,2020-01-01,FDK,12138,1577916300,1577916420,1577916960,1577917080,32,0,660,660
2,2020-01-01,KKP,12138,1577917380,1577917500,1577918220,1577918280,45,0,840,780


In [417]:
batch = 32
learnning_rate = 0.02

In [418]:
#train currently at sn - tnA , tnD
#next station - sn+1  sn-1
#train Characterstics - X1 = 1
#Departure Delay time at sn - X2 = tanD - tnD
#Arrival Delay time at sn - X3 = tanA- tnA
#Planning running time of train in last section - tan

In [419]:
# df.columns

In [420]:
def timeseries_data(time_steps,dfi):
    dfx=dfi
    columns = dfx.columns
    trains = list(set(dfx.trainCode_n.tolist()))
    train_dfs = []
    dfs = dfx.groupby("trainCode_n")
    for train in trains :
        per_train_df=dfs.get_group(train)
#         print(per_train_df.shape)        
        for i in range(1,2):

    #   Only for 1 - upcoming station
            for col in columns:
    #             print(j)
                if col not in ['trainCode_n']:
                    per_train_df[str(col)+"+{}".format(i)] = per_train_df[str(col)].shift(-i)
        
        #for previous stations logic
        for i in range(1,time_steps+1):
            for col in columns:
    #             print(j)
                if col not in ['trainCode_n']:
                    per_train_df[str(col)+"-{}".format(i)] = per_train_df[str(col)].shift(i)
        
        per_train_df=per_train_df[time_steps:-1]
        
        #distance_n is the amount of distance that the train 
        #has to travel to reach next station
        per_train_df['distance_n-1']=per_train_df['distance_n']-per_train_df['distance_n-1']
        per_train_df['distance_n']=per_train_df['distance_n+1']-per_train_df['distance_n']

        #planned running is the amount of time that 
        #the train is scheduled for to get to next station from current station 
        per_train_df['planned_running_time_n-1'] = per_train_df['scheduledArrival_n'] - per_train_df['scheduledDeparture_n-1']
        per_train_df['planned_running_time_n'] = per_train_df['scheduledArrival_n+1'] - per_train_df['scheduledDeparture_n']

        del per_train_df['actualArrival_n']
        del per_train_df['scheduledArrival_n']
        del per_train_df['scheduledDeparture_n']
        del per_train_df['actualDeparture_n']
        del per_train_df['runDate']
        del per_train_df['dayCount_n']
        del per_train_df['stations_n']
        
        del per_train_df['actualArrival_n-1']
        del per_train_df['scheduledArrival_n-1']
        del per_train_df['scheduledDeparture_n-1']
        del per_train_df['actualDeparture_n-1']
        del per_train_df['dayCount_n-1']
        del per_train_df['runDate-1']
        del per_train_df['stations_n-1']

        del per_train_df['actualArrival_n+1']
        del per_train_df['scheduledArrival_n+1']
        del per_train_df['scheduledDeparture_n+1']
        del per_train_df['actualDeparture_n+1']
        del per_train_df['runDate+1']
        del per_train_df['dayCount_n+1']
        del per_train_df['stations_n+1']
        
        del per_train_df['trainCode_n']
        del per_train_df['distance_n+1']

#         return per_train_df
#         print(per_train_df.columns)
        train_dfs.append(per_train_df)
        
    final_df=pd.concat(train_dfs)   
    
    return final_df

In [421]:
#FINAL ONE
def timeseries_data2(time_steps,dfi):
    count=0
    dfx=dfi
    columns = dfx.columns
    print(columns)
    trains = list(set(dfx.trainCode_n.tolist()))
    train_dfs = []
    dfs = dfx.groupby("trainCode_n")
    for train in trains :
        train_num_df=dfs.get_group(train)
        distinct_train_dfs= train_num_df.groupby('runDate')
        for index,group in distinct_train_dfs:
            per_train_df = group
#         print(per_train_df.shape)        
            for i in range(1,2):

        #   Only for 1 - upcoming station
                for col in columns:
        #             print(j)
                    if col not in ['trainCode_n']:
                        per_train_df[str(col)+"+{}".format(i)] = per_train_df[str(col)].shift(-i)

            #for previous stations logic
            for i in range(1,time_steps+1):
                for col in columns:
        #             print(j)
                    if col not in ['trainCode_n']:
                        per_train_df[str(col)+"-{}".format(i)] = per_train_df[str(col)].shift(i)
            
            per_train_df=per_train_df[time_steps:-1]

            #distance_n is the amount of distance that the train 
            #has to travel to reach next station
            per_train_df['distance_n-1']=per_train_df['distance_n']-per_train_df['distance_n-1']
            per_train_df['distance_n']=per_train_df['distance_n+1']-per_train_df['distance_n']

            #planned running is the amount of time that 
            #the train is scheduled for to get to next station from current station
            per_train_df['planned_running_time_n-1'] = per_train_df['scheduledArrival_n'] - per_train_df['scheduledDeparture_n-1']
            for i in range(time_steps,1,-1):
                per_train_df['planned_running_time_n-{}'.format(i)] = per_train_df['scheduledArrival_n-{}'.format(i-1)] - per_train_df['scheduledDeparture_n-{}'.format(i)]
            per_train_df['planned_running_time_n'] = per_train_df['scheduledArrival_n+1'] - per_train_df['scheduledDeparture_n']

            del per_train_df['actualArrival_n']
            del per_train_df['scheduledArrival_n']
            del per_train_df['scheduledDeparture_n']
            del per_train_df['actualDeparture_n']
            del per_train_df['runDate']
            del per_train_df['dayCount_n']
            del per_train_df['stations_n']

            for i in range(1,time_steps+1):
                del per_train_df['actualArrival_n-{}'.format(i)]
                del per_train_df['scheduledArrival_n-{}'.format(i)]
                del per_train_df['scheduledDeparture_n-{}'.format(i)]
                del per_train_df['actualDeparture_n-{}'.format(i)]
                del per_train_df['dayCount_n-{}'.format(i)]
                del per_train_df['runDate-{}'.format(i)]
                del per_train_df['stations_n-{}'.format(i)]

            del per_train_df['actualArrival_n+1']
            del per_train_df['scheduledArrival_n+1']
            del per_train_df['scheduledDeparture_n+1']
            del per_train_df['actualDeparture_n+1']
            del per_train_df['runDate+1']
            del per_train_df['dayCount_n+1']
            del per_train_df['stations_n+1']

            del per_train_df['trainCode_n']
            del per_train_df['distance_n+1']

#             return per_train_df
    #         print(per_train_df.columns)
            train_dfs.append(per_train_df)
        
    final_df=pd.concat(train_dfs)   
    
    return final_df

In [422]:
dfnew = timeseries_data2(1,df)

Index(['runDate', 'stations_n', 'trainCode_n', 'scheduledArrival_n',
       'scheduledDeparture_n', 'actualArrival_n', 'actualDeparture_n',
       'distance_n', 'dayCount_n', 'ArrivalDelay_n', 'DepartureDelay_n'],
      dtype='object')


In [423]:
dfnew.columns

Index(['distance_n', 'ArrivalDelay_n', 'DepartureDelay_n', 'ArrivalDelay_n+1',
       'DepartureDelay_n+1', 'distance_n-1', 'ArrivalDelay_n-1',
       'DepartureDelay_n-1', 'planned_running_time_n-1',
       'planned_running_time_n'],
      dtype='object')

In [424]:
# del dfnew['day_n+1']
# del dfnew['hour_n+1']

In [425]:
from sklearn.preprocessing import RobustScaler
def get_training_data(modelled_df):
    print(modelled_df)
    target_columns=['ArrivalDelay_n+1','DepartureDelay_n+1']
    y=modelled_df[target_columns]
    X=modelled_df.drop(target_columns,axis=1,inplace=False)
#     X=modelled_df[~target_columns]

    #of shape - (train_samples,time_steps,features)
    
    rs1=RobustScaler()
    rs2=RobustScaler()
    X=rs1.fit_transform(X)
    print(X.shape)  
    X=np.array(X).reshape(X.shape[0],2,4)
   
    y=np.array(y)
    y=rs2.fit_transform(y)
#     y=rs.transform(y)
    
    return (X,y)

In [426]:
# X;y.shape

In [427]:
X,y=get_training_data(dfnew)

       distance_n  ArrivalDelay_n  DepartureDelay_n  ArrivalDelay_n+1  \
8284         48.0             180               120            -300.0   
8285         15.0            -300                60             -60.0   
8286         24.0             -60                 0             420.0   
8287         19.0             420               420             540.0   
8288         17.0             540               420             480.0   
...           ...             ...               ...               ...   
31867        43.0            4860              4860            4140.0   
31869        36.0            4140              4380            4800.0   
31870        16.0            4800              4800            5220.0   
31871         5.0            5220              5220            4920.0   
31872        36.0            4920              5520            5280.0   

       DepartureDelay_n+1  distance_n-1  ArrivalDelay_n-1  DepartureDelay_n-1  \
8284                 60.0          95.0   

In [428]:
# X

In [429]:
print(X.shape)
print(y.shape)

(29715, 2, 4)
(29715, 2)


In [430]:
# X

In [431]:
y

array([[-0.57142857, -0.45238095],
       [-0.47619048, -0.47619048],
       [-0.28571429, -0.30952381],
       ...,
       [ 1.61904762,  1.5952381 ],
       [ 1.5       ,  1.71428571],
       [ 1.64285714,  1.61904762]])

In [432]:
#Validation split
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.20,shuffle=True,random_state=42)
print(X_train.shape)
print(y_train.shape)

(23772, 2, 4)
(23772, 2)


In [433]:
from keras.layers import LSTM,LSTMCell,Dense
from keras.models import Sequential
from keras.optimizers import SGD
from keras.models import Model
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [434]:
def create_lstm_model(train_X):
    model = Sequential()
    model.add(LSTM(64, input_shape=(train_X.shape[1],train_X.shape[2])))
    model.add(Dense(10))
    model.add(Dense(2))
#     model.compile()
    return model

In [435]:
model1 = create_lstm_model(X_train)
model1.summary()
opt=SGD(learning_rate=0.02)
model1.compile(loss='mae', optimizer=opt)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 64)                17664     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 22        
Total params: 18,336
Trainable params: 18,336
Non-trainable params: 0
_________________________________________________________________


In [436]:
history = model1.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_valid, y_valid), verbose=2, shuffle=True)

Epoch 1/100
372/372 - 3s - loss: 0.3531 - val_loss: 0.2611
Epoch 2/100
372/372 - 2s - loss: 0.2591 - val_loss: 0.2523
Epoch 3/100
372/372 - 2s - loss: 0.2508 - val_loss: 0.2434
Epoch 4/100
372/372 - 2s - loss: 0.2440 - val_loss: 0.2353
Epoch 5/100
372/372 - 2s - loss: 0.2375 - val_loss: 0.2369
Epoch 6/100
372/372 - 2s - loss: 0.2323 - val_loss: 0.2215
Epoch 7/100
372/372 - 2s - loss: 0.2272 - val_loss: 0.2178
Epoch 8/100
372/372 - 2s - loss: 0.2221 - val_loss: 0.2382
Epoch 9/100
372/372 - 2s - loss: 0.2178 - val_loss: 0.2510
Epoch 10/100
372/372 - 2s - loss: 0.2136 - val_loss: 0.2259
Epoch 11/100
372/372 - 2s - loss: 0.2093 - val_loss: 0.1987
Epoch 12/100
372/372 - 2s - loss: 0.2042 - val_loss: 0.2063
Epoch 13/100


KeyboardInterrupt: 

## Inferencing using trained model on test set

In [437]:
dfnew.columns

Index(['distance_n', 'ArrivalDelay_n', 'DepartureDelay_n', 'ArrivalDelay_n+1',
       'DepartureDelay_n+1', 'distance_n-1', 'ArrivalDelay_n-1',
       'DepartureDelay_n-1', 'planned_running_time_n-1',
       'planned_running_time_n'],
      dtype='object')

In [438]:
print("s")

s


In [439]:
trains = list(set(df.trainCode_n.tolist()))
c={}
for i in trains:
    dfp = df[df['trainCode_n']==i]
    stations = dfp.groupby('runDate').first().shape[0]
    c[str(i)]=stations
c

{'12321': 47,
 '12833': 40,
 '12834': 45,
 '12295': 46,
 '12616': 47,
 '12617': 47,
 '12138': 49,
 '12139': 45,
 '12809': 44,
 '12840': 47,
 '12656': 50,
 '12625': 50,
 '12722': 43,
 '12311': 49,
 '12925': 42}

In [440]:
#FINAL ONE
def create_test_df(time_steps,dfi):
    count=0
    dfx=dfi
    columns = dfx.columns
    print(columns)
    trains = list(set(dfx.trainCode_n.tolist()))
    print(trains)
    train_dfs = []
    dfs = dfx.groupby("trainCode_n")
    for train in trains :
#         print(train.shape)
        train_num_df=dfs.get_group(train)
        distinct_train_dfs= train_num_df.groupby('runDate')
        for index,group in distinct_train_dfs:
            per_train_df = group

            #for previous stations logic
            for i in range(1,time_steps+1):
                for col in columns:
        #             print(j)
                    if col not in ['trainCode_n']:
                        per_train_df[str(col)+"-{}".format(i)] = per_train_df[str(col)].shift(i)
            
            per_train_df=per_train_df[time_steps:-1]

            #distance_n is the amount of distance that the train 
            #has to travel to reach next station
            per_train_df['distance_n-1']=per_train_df['distance_n']-per_train_df['distance_n-1']

            #planned running is the amount of time that 
            #the train is scheduled for to get to next station from current station
        return per_train_df

In [441]:
temp_df=df2
temp_dfs = temp_df.groupby('trainId')
for i,train_df in temp_dfs:
    trainCode = list(set(train_df['trainCode'].tolist()))[0]
    trainId = list(set(train_df['trainId'].tolist()))[0]
    print("predicting for train Id {}".format(trainId))
    to_append_df = df[df['trainCode_n']==trainCode]
    to_append_df = to_append_df[-c[str(trainCode)]:]

    
#     print(to_append_df.columns)
    
#     shape_of_appended= int(c[str(trainCode)])
    last_two_train = to_append_df.tail(2)
    print(last_two_train.columns)
#     print(last_departure_delay)
#     del train_df['day']
    
#     train_df.at[0,'Actual_']
    
    train_df['stations_n']=train_df['stations']
    del train_df['stations']
    del train_df['trainId']
    train_df['trainCode_n']=train_df['trainCode']
    del train_df['trainCode']
    del train_df['index']
    del train_df['trainStationId']
    del train_df['day']
    train_df['scheduledArrival_n']=train_df['scheduledArrival']
    train_df['scheduledDeparture_n']=train_df['scheduledDeparture']
    train_df['distance_n']=train_df['distance']
    train_df['dayCount_n']=train_df['dayCount']
    
    
    del train_df['scheduledArrival']
    del train_df['scheduledDeparture']
    del train_df['dayCount']
    del train_df['distance']

    print(train_df.columns)
#     print(group.shape)


predicting for train Id 11528
Index(['runDate', 'stations_n', 'trainCode_n', 'scheduledArrival_n',
       'scheduledDeparture_n', 'actualArrival_n', 'actualDeparture_n',
       'distance_n', 'dayCount_n', 'ArrivalDelay_n', 'DepartureDelay_n'],
      dtype='object')
Index(['runDate', 'stations_n', 'trainCode_n', 'scheduledArrival_n',
       'scheduledDeparture_n', 'distance_n', 'dayCount_n'],
      dtype='object')
predicting for train Id 11529
Index(['runDate', 'stations_n', 'trainCode_n', 'scheduledArrival_n',
       'scheduledDeparture_n', 'actualArrival_n', 'actualDeparture_n',
       'distance_n', 'dayCount_n', 'ArrivalDelay_n', 'DepartureDelay_n'],
      dtype='object')
Index(['runDate', 'stations_n', 'trainCode_n', 'scheduledArrival_n',
       'scheduledDeparture_n', 'distance_n', 'dayCount_n'],
      dtype='object')
predicting for train Id 11615
Index(['runDate', 'stations_n', 'trainCode_n', 'scheduledArrival_n',
       'scheduledDeparture_n', 'actualArrival_n', 'actualDeparture_

In [442]:
test_df  =create_test_df(1,train_df)

Index(['runDate', 'stations_n', 'trainCode_n', 'scheduledArrival_n',
       'scheduledDeparture_n', 'distance_n', 'dayCount_n'],
      dtype='object')
[12311]


In [443]:
dfnew.columns

Index(['distance_n', 'ArrivalDelay_n', 'DepartureDelay_n', 'ArrivalDelay_n+1',
       'DepartureDelay_n+1', 'distance_n-1', 'ArrivalDelay_n-1',
       'DepartureDelay_n-1', 'planned_running_time_n-1',
       'planned_running_time_n'],
      dtype='object')

In [444]:
test_df.columns
del test_df['runDate-1']
del test_df['stations_n-1']
del test_df['stations_n']
del test_df['runDate']
del test_df['dayCount_n-1']
del test_df['dayCount_n']

In [445]:
test_df.columns

Index(['trainCode_n', 'scheduledArrival_n', 'scheduledDeparture_n',
       'distance_n', 'scheduledArrival_n-1', 'scheduledDeparture_n-1',
       'distance_n-1'],
      dtype='object')

In [446]:
#Iterating and Recursively doing the predictions
for i in range(0,51):
    X_test=test_df[i:i+2]
#     model1.predict(X_test)

for i in range()

### Comparing this with baseline mean delays

In [447]:
# for i in list(set(df.trainCode_n.tolist())):
temp1 = df[['trainCode_n','ArrivalDelay_n','DepartureDelay_n','stations_n']]
t=temp1.groupby('trainCode_n')
    

    

In [448]:
# t.get_group(12833)

In [461]:
lists = []
for i in list(set(df2.trainId.tolist())):
    t2 = df2[df2['trainId']==i]
    t2['stations_n']=t2['stations']
    t2 = t2[:-1]
    code = int(list(set(t2.trainCode.tolist()))[0])
#     st=t2.stations.tolist()
    
    delays = t.get_group(code).groupby('stations_n').mean().reset_index()
    print
    t2=pd.merge(t2,delays,on='stations_n')
    
#     print(delays.columns)
    print(t2.columns)
    lists.append(t2)

Index(['runDate', 'stations', 'trainId', 'trainCode', 'index',
       'trainStationId', 'scheduledArrival', 'scheduledDeparture', 'distance',
       'dayCount', 'day', 'stations_n', 'trainCode_n', 'ArrivalDelay_n',
       'DepartureDelay_n'],
      dtype='object')
Index(['runDate', 'stations', 'trainId', 'trainCode', 'index',
       'trainStationId', 'scheduledArrival', 'scheduledDeparture', 'distance',
       'dayCount', 'day', 'stations_n', 'trainCode_n', 'ArrivalDelay_n',
       'DepartureDelay_n'],
      dtype='object')
Index(['runDate', 'stations', 'trainId', 'trainCode', 'index',
       'trainStationId', 'scheduledArrival', 'scheduledDeparture', 'distance',
       'dayCount', 'day', 'stations_n', 'trainCode_n', 'ArrivalDelay_n',
       'DepartureDelay_n'],
      dtype='object')
Index(['runDate', 'stations', 'trainId', 'trainCode', 'index',
       'trainStationId', 'scheduledArrival', 'scheduledDeparture', 'distance',
       'dayCount', 'day', 'stations_n', 'trainCode_n', 'Arrival

In [462]:
    
final_df = pd.concat(lists)

In [467]:
final_df['ArrivalDelay_n']=final_df['ArrivalDelay_n'].apply(lambda x: x/60)
final_df['DepartureDelay_n']=final_df['DepartureDelay_n'].apply(lambda x: x/60)

In [468]:
final_baseline_df = final_df

In [470]:
# final_baseline_df

In [466]:
final_baseline_df.to_csv("AnubhavKesari_solutions.csv",index=False)